In [ ]:
%load_ext autoreload
%autoreload 2

from setup_imports import *  # noqa: F401,F403
from src.phrases.phrase_model import Phrase, Translation, get_phrase, get_phrase_by_english
from src.models import BCP47Language
from src.connections.gcloud_auth import get_firestore_client

In [ ]:
# Step 1: Create a new phrase
phrase = Phrase.create_phrase("She runs to the store daily", source="manual")
print(f"Created phrase: {phrase.english}")
print(f"Phrase hash: {phrase.phrase_hash}")
print(f"Phrase model dump:\n{phrase.model_dump()}")

In [ ]:
# Step 2: Translate the phrase to French
fr_translation = phrase.translate(BCP47Language.get("fr-FR"), refine=False)
print(f"French translation: {fr_translation.text}")
print(f"Language: {fr_translation.language.to_tag()}")
print(f"Tokens: {fr_translation.tokens}")
print(f"Audio: {fr_translation.audio}")  # Should be None initially

In [ ]:
# Step 3: Add another translation (German) and verify translations list
de_translation = phrase.translate(BCP47Language.get("de-DE"), refine=False)
print(f"German translation: {de_translation.text}")
print(f"\nTotal translations on phrase: {len(phrase.translations)}")
for trans in phrase.translations:
    print(f"  - {trans.language.to_tag()}: {trans.text}")

In [ ]:
# Step 4: Upload phrase to Firestore
firestore_client = get_firestore_client("firephrases")
uploaded_hash = phrase.upload_phrase(firestore_client)
print(f"Uploaded phrase with hash: {uploaded_hash}")

In [ ]:
# Step 5: Retrieve phrase by hash
retrieved_phrase = get_phrase(uploaded_hash)
print(f"Retrieved phrase: {retrieved_phrase.english}")
print(f"Phrase hash: {retrieved_phrase.phrase_hash}")
print(f"Source: {retrieved_phrase.source}")
print(f"Translations: {len(retrieved_phrase.translations)}")
for trans in retrieved_phrase.translations:
    print(f"  - {trans.language.to_tag()}: {trans.text}")
print(f"\nRetrieved phrase model_dump:\n{retrieved_phrase.model_dump()}")

In [ ]:
# Step 6: Retrieve phrase by English text (convenience wrapper)
english_phrase = "She runs to the store daily"
retrieved_by_english = get_phrase_by_english(english_phrase)
print(f"Retrieved by English text: {retrieved_by_english.english}")
print(f"Hash: {retrieved_by_english.phrase_hash}")
print(f"Translations: {len(retrieved_by_english.translations)}")
for trans in retrieved_by_english.translations:
    print(f"  - {trans.language.to_tag()}: {trans.text}")

In [ ]:
# Step 7: Verify data integrity - compare original and retrieved
print("=== Data Integrity Check ===")
print(f"English matches: {phrase.english == retrieved_phrase.english}")
print(f"Hash matches: {phrase.phrase_hash == retrieved_phrase.phrase_hash}")
print(f"Translation count matches: {len(phrase.translations) == len(retrieved_phrase.translations)}")

# Verify each translation
original_translations = {t.language.to_tag(): t.text for t in phrase.translations}
retrieved_translations = {t.language.to_tag(): t.text for t in retrieved_phrase.translations}

print(f"\nOriginal translations: {original_translations}")
print(f"Retrieved translations: {retrieved_translations}")
print(f"Translations match: {original_translations == retrieved_translations}")